# Finding Cu crystal ground state via the Augmented Plane method

Adapted from Thijssen

In [1]:
import numpy as np
from scipy import special as sp

In [7]:
# constants
a = 6.822 #au
R = 1
V = 3*(a**3)/4

nLim = 3 

b1 = 2*np.pi*np.array([-1,1,1])/a
b2 = 2*np.pi*np.array([1,-1,1])/a
b3 = 2*np.pi*np.array([1,1,-1])/a
Ks =[l*b1 + m*b2 + n*b3 for l in range(-nLim,nLim+1) for m in range(-nLim,nLim+1) for n in range(-nLim,nLim+1)]



In [19]:
#generate A
def kroneckerDelta(i,j):
    if i == j:
        return 1
    return 0

A = np.empty([len(Ks),len(Ks)])
for i,K_i in enumerate(Ks):
    for j,K_j in enumerate(Ks):
        kDiff = np.linalg.norm(K_i-K_j)
        if kDiff == 0:
            A[i,j] = 1
        else:
            A[i,j] = -4*np.pi*(R**2)*sp.spherical_jn(1,kDiff*R)/(V*kDiff)

print(A)




[[ 1.00000000e+00 -1.35026863e-02 -5.09872972e-03 ... -6.24728351e-04
  -6.24843489e-04 -5.61053190e-04]
 [-1.35026863e-02  1.00000000e+00 -1.35026863e-02 ... -6.24843489e-04
  -6.24728351e-04 -6.24843489e-04]
 [-5.09872972e-03 -1.35026863e-02  1.00000000e+00 ... -5.61053190e-04
  -6.24843489e-04 -6.24728351e-04]
 ...
 [-6.24728351e-04 -6.24843489e-04 -5.61053190e-04 ...  1.00000000e+00
  -1.35026863e-02 -5.09872972e-03]
 [-6.24843489e-04 -6.24728351e-04 -6.24843489e-04 ... -1.35026863e-02
   1.00000000e+00 -1.35026863e-02]
 [-5.61053190e-04 -6.24843489e-04 -6.24728351e-04 ... -5.09872972e-03
  -1.35026863e-02  1.00000000e+00]]


In [ ]:
#shall use hydrogen atom approximation as all other electrons have a shielding effect (this is rough)

In [ ]:
def getH(k):
    #set up k dependent matricies 
    B = np.empty([len(Ks),len(Ks)])
    C = np.empty([len(Ks),len(Ks),len(Ks)])
    qs = [k + K for K in Ks]

    #find B and C
    for i in range(len(Ks)):
        for j in range(len(Ks)):
            qDot = np.dot(qs[i],qs[j])
            B[i,j] = A[i,j]*qDot/2
            qi = np.linalg.norm(qs[i])
            qj = np.linalg.norm(qs[j])
            for l,K_l in enumerate(Ks):
                if qi == 0 or qj == 0:
                    C[i,j,l] = 0
                else: 
                    C[i,j,l] = (2*l + 1)*2*np.pi*(R**2)*sp.eval_legendre(l,qDot/(qi*qj))*sp.spherical_jn(l,qi*R)*sp.spherical_jn(l,qj*R)/V
    
    #find H with these matricies


